In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=8975f9e151bc1cd6ae821740842283e74f43247ba2c21d015a1230617df30974
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 51.6 MB/

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
#load pdf files
loader = PyPDFDirectoryLoader("/content/sample_data/Data/")
data = loader.load()

In [ ]:
print(data)

[Document(page_content='РОССИЙСКАЯ ФЕДЕРАЦИЯ\nУголовный кодекс Российской Федерации\n(с изменениями на 29  декабря 2022 года)\n(редакция, действующая с 1 января 2023 года)\nДокумент с изменениями, внесенными:\nФедеральным законом от 27 мая 1998 года N 77-ФЗ (Российская газета, N 103, 30.05.98);\nФедеральным законом от 25 июня 1998 года N 92-ФЗ (Российская газета, N 120, 27.06.98);\nФедеральным законом от 9 февраля 1999 года N 24-ФЗ (Российская газета, N 27, 12.02.99);\nФедеральным законом от 9 февраля 1999 года N 26-ФЗ (Российская газета, N 27, 12.02.99);\nФедеральным законом от 15 марта 1999 года N 48-ФЗ (Российская газета, N 50, 17.03.99);\nФедеральным законом от 18 марта 1999 года N 50-ФЗ (Российская газета, N 54, 23.03.99);\nФедеральным законом от 9 июля 1999 года N 156-ФЗ (Российская газета, N 134, 14.07.99);\nФедеральным законом от 9 июля 1999 года N 157-ФЗ (Российская газета, N 134, 14.07.99);\nФедеральным законом от 9 июля 1999 года N 158-ФЗ (Российская газета, N 134, 14.07.99)

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [ ]:
len(text_chunks)

323

In [ ]:
#get the third chunk
text_chunks[20]

Document(page_content='охватывались\n \nего\n \nумыслом.\n \nДругие\n \nучастники\n \nорганизованной\n \nгруппы\n \nили\n \nпреступного\n \nсообщества\n(\nпреступной\n \nорганизации\n) \nнесут\n \nуголовную\n \nответственность\n \nза\n \nучастие\n \nв\n \nних\n \nв\n \nслучаях\n, \nпредусмотренных\nстатьями\n 205_4, 208, 209, 210 \nи\n 282_1 \nнастоящего\n \nКодекса\n, \nа\n \nтакже\n \nза\n \nпреступления\n, \nв\n \nподготовке\n \nили\nсовершении которых они участвовали.\n(\nЧасть\n \nв\n \nредакции\n, \nвведенной\n \nв\n \nдействие\n \nс\n 17 \nноября\n 2009 \nгода\n \nФедеральным\n \nзаконом\n \nот\n 3 \nноября\n 2009\nгода\n N 245-\nФЗ\n; \nв\n \nредакции\n, \nвведенной\n \nв\n \nдействие\n \nс\n 14 \nноября\n 2013 \nгода\n \nФедеральным\n \nзаконом\n \nот\n 2 \nноября\n2013 года N 302-ФЗ. - См. предыдущую редакцию)\n6. \nСоздание\n \nорганизованной\n \nгруппы\n \nв\n \nслучаях\n, \nне\n \nпредусмотренных\n \nстатьями\n \nОсобенной\n \nчасти\nнастоящего\n \nКодекса\n, \nвлечет\n \n

In [ ]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
#connect to google drive
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
qa = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))
query = "Hello"
qa.run(query)

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Llama.generate: prefix-match hit


Answer:  It is a statistical method used to analyze the linear relationship between two or more variables. In this case, it is used to estimate the coefficients of a linear regression model.


Llama.generate: prefix-match hit


Answer:  sandwich provides various functions for estimating the covariance matrix. The most commonly used ones are vcovHAC and vcovHC. These functions can take different weighting schemes, including kernel-based HAC estimation with automatic bandwidth selection based on weightsAndrews andbwAndrews. Other functions like weightsLumley provide different weighting schemes, such as truncated and smoothed weights, which are useful in certain applications. In econometric analyses, these functions are used to compute partial t-tests for assessing the significance of a parameter. The choice of estimator depends on the presence of heteroscedasticity and autocorrelation in the errors.
